# Collaborative Filtering Recommendation Algorithm
## Div Dasani

This algorithm employs the ALS implicit feedback approach to collaborative filtering descibed by [Hu et al.](http://yifanhu.net/PUB/cf.pdf) on Instacart order data to recommend products to users.

## A. Setup
Instructions: In order to run this notebook, make sure you have all of the below packages installed. In particular, *implicit* can be installed via the command:
```bash
conda install -c conda-forge implicit
```
Additionally, please make sure [this](https://www.instacart.com/datasets/grocery-shopping-2017) Instacart data is downloaded and placed in the same directory as the notebook.

In [1]:
import pandas as pd
import numpy as np
import scipy.sparse as sparse
import random
import implicit

## B. Hyperparameters
Below are the hyperparameters that govern the algorithm. They will be referenced throughout the notebook as necessary.
<br>
*train_ratio*- denotes the percentage of data that will be utilized for training
<br>
*factors*- denotes the number of latent factors assumed by the algorithm
<br>
*regularization*- denotes the magnitude of the regularization parameter, $\lambda$
<br>
*iterations*- denotes the number of iterations run by the algorithm
<br>
*alpha*- denotes the learning rate of confidence of the algorithm
<br>
*N*- denotes the recommendation list size for calculating the *Mean Percentile Ranking (MPR)*

In [14]:
train_ratio = 0.7
factors = 20
regularization = 0.1
iterations = 20
alpha = 15
N = 10**3

## C. Data Cleansing
First, the data is loaded into the notebook as Pandas DataFrames. Then, the *user_id* values are joined with the *product_id* values into one DataFrame through the *order_id* key. After this, a *count* column is added, and identical instances of <*user_id*, *product_id*> are grouped together by incrementing the count of the row. Thus, all <*user_id*, *product_id*> tuples in the DataFrame are unique. Finally, the data is split into its training and testing components. A product dictionary DataFrame mapping product ids to product descriptions is also created for returning the names of recommended products.

In [3]:
product_dic = pd.read_csv('products.csv')
product_dic = product_dic[['product_id','product_name']]

df_users = pd.read_csv('orders.csv')
df_users = df_users[df_users['eval_set'] == 'prior']
df_users = df_users[['order_id','user_id']]

df_products = pd.read_csv('order_products__prior.csv')
df_products = df_products[['order_id','product_id']]

df_merged = pd.merge(df_users, df_products, on='order_id', how='left')
df_merged = df_merged[['user_id','product_id']]
df_merged['count'] = 1
df_merged = df_merged.groupby(['user_id','product_id'], as_index=False)['count'].sum()

df_train = df_merged.sample(frac=train_ratio)
df_test = (df_merged.merge(df_train, on=['user_id','product_id'], how='left', indicator=True)
           .query('_merge == "left_only"').drop('_merge', 1))
df_test = df_test.drop(['count_y'],axis=1)
df_test.columns = ['user_id', 'product_id', 'count_purchase']

## D. The Model
There are two types of data one can employ when making recommendations: explicit and implicit data. The former is feedback provided directly by the user that informs us of her opinion of the product (i.e. ratings). The latter is information that is indirectly generated by the user and may possess some correlation to the user's opinion of a product. This dataset is implicit, as we are using the number of times a user purchases a product to inform us of her preferences.
<br>
<br>
Thus, our data can be represented as a matrix $R\inℝ^{u*i}$ whose rows denote users ($u$) and whose columns denote products ($i$). The values of each cell in the matrix represent the number of times a particular product is purchased by a particular user, $r_{u,i}$. Because there are thousands of products and each user only purchases a fraction of these, $R$ is highly sparse. The way the algorithm learns from this missing data is through factorizing this data into two matrices $X\inℝ^{u*f}$ and $Y\inℝ^{f*i}$ where $f$ represents *factors*, the number of latent factors. Thus, each value in $X$ is a weight for a feature for each user and each value in $Y$ is a weight for a feature for each product. The goal is to tune the weights in each of these matrices such that $X^TY\approx R$.
<br>
<br>
The algorithm that tunes these weights is the *Alternating Least Squares (ALS)* algorithm. This algorithm works by alternatively tuning each matrix $X$ and $Y$ while holding the other constant. The algorithm is defined as,
<center>$\min\limits_{X*,Y*}\sum_{u,i} [c_{u,i}(p_{u,i} - X_{u}^TY_{i})^2] + \lambda(\sum_{u}||X_{u}||^2 + \sum_{i}||Y_{i}||^2)$</center>
where $p$ is the preference function given by,
<center>$$ p_{u,i}(r_{u,i})=   \left\{
\begin{array}{ll}
      1 & r_{u,i}>0 \\
      0 & r_{u,i}=0 \\
\end{array} 
\right.  $$</center>
$c$ is the confidence function with learning rate *alpha* given by,
<center>$c_{u,i} = 1 + \alpha*r_{u,i}$</center>
and $\lambda$ represents the regularization parameter *regularization*, used to prevent overfitting by penalizing heavy weights.

Below the model is implemented as described above. First, two sparse matrices are created: the item-user matrix is used to train the algorithm, while the user-item matrix will be used to extract recommendations for a given user from the trained model.The former matrix is multiplied by the confidence learning rate *alpha* before being fed to the ALS algorithm with prespecified parameters *factors* and *regularization* as described above, and minimizes the loss function over a prespecified number of iterations *iterations*. This notebook employs the implicit package to implement the ALS algorithm due to its ability to run calculations in C++, vastly increasing computational efficiency.

In [5]:
def build_model(factors,regularization,iterations,alpha):
    sparse_item_user = sparse.csr_matrix((df_train['count'].astype(float), (df_train['product_id'], df_train['user_id'])))
    sparse_user_item = sparse.csr_matrix((df_train['count'].astype(float), (df_train['user_id'], df_train['product_id'])))
    data_conf = (sparse_item_user * alpha).astype('double')
    
    model = implicit.als.AlternatingLeastSquares(factors=factors, regularization=regularization, iterations=iterations)
    model.fit(data_conf)
    
    return model, sparse_user_item

In [10]:
model, sparse_user_item = build_model(factors,regularization,iterations,alpha)

100%|████████████████████████████████████████████████████████████████████████████████| 20.0/20 [01:35<00:00,  4.70s/it]


## E. Output Functions
The *recommendations* method employs the user-item matrix along with a user id to recommend N products that the trained model believes the user has the highest chance of purchasing.
<br>
The *similar_products* method takes in a product id and returns N products that are the most similar to the product based on the trained model.
<br>
The *name* method takes in a list of product ids and returns the names of the products.

In [6]:
def name(product_ids):
    product_names = []
    for product_id in product_ids:
        product_names.append(product_dic.product_name.loc[product_dic.product_id == product_id].iloc[0])
    return pd.DataFrame({'product': product_names})

In [7]:
def similar_products(product_id, N, named=False):
    similar = model.similar_items(product_id, N+1)
    
    recs = []
    for item in similar[1:]:
        recs.append(item[0])
    if named:
        return name(recs)
    else:
        return recs

In [8]:
def recommendations(user_id, N, named=False):
    recommended = model.recommend(user_id, sparse_user_item, N=N)
    
    recs = []
    for item in recommended:
        recs.append(item[0])
    if named:
        return name(recs)
    else:
        return recs

## F. Predictive Ability
Now that the model has been trained, we will analyze its predictive ability. Below is an instance of a single user, whose purchase history is illustrated. This user tends to purchase health-oriented items, such as "Organic Quick Oats" and "Michigan Organic Kale". As a result, the algorithm recommends items in this category, such as "Organic Riced Cauliflower" and "Organic Spaghetti Squash". The recommendation system also nudges the user to products that are incredibly similar to previously purchased products; for example, the user purchased "Dairy Free Unsweetened Vanilla Coconut Milk", so the algorithm recommended "Organic Unsweetened Vanilla Almond Milk".

In [12]:
user_id = 202279

print('User Purchase History')
print(name(df_merged[df_merged['user_id'] == user_id].product_id.tolist()))
print('\nRecommended Products')
print(recommendations(user_id,5,named=True))

User Purchase History
                                            product
0                 Vanilla Almond Breeze Almond Milk
1                         Organic Turkey Bone Broth
2          All Natural No Stir Creamy Almond Butter
3                      Sustainably Soft Bath Tissue
4               Homestyle Savory Chicken Bone Broth
5                   100% Organic Raw Coconut Butter
6        Organic Raw Unfiltered Apple Cider Vinegar
7         Renew Life Total Body 7-Day Rapid Cleanse
8                 Natural Calm Magnesium Supplement
9              All Natural Cocoa Powder Unsweetened
10                 Cilantro Avocado Yogurt Dressing
11   Dressing & Sandwich Spread Made with Olive Oil
12                                      Dry Shampoo
13                                    Garlic Powder
14                   Italian Extra Virgin Olive Oil
15                      Nutritional Yeast Seasoning
16              French Vanilla Coconut Milk Creamer
17             Grade A Large Eggs Cage Fre

The algorithm is also able to understand what products are similar to one another. In the example below, the algorithm is provided with "Chocolate Sandwich Cookies" and returns several similar snacks, such as "Bite Size Candies, Original" and "Cheez-It Cheddar Cracker".

In [19]:
target = "Chocolate Sandwich Cookies"

target_id = product_dic.product_id.loc[product_dic.product_name == target].iloc[0]
print('Given Product: {}'.format(target))
print('\nSimilar Products:')
print(similar_products(target_id,5,named=True))

Given Product: Chocolate Sandwich Cookies

Similar Products:
                       product
0   Cup Noodles Chicken Flavor
1  Bite Size Candies, Original
2                Juice Squeeze
3  Original Snack Mix 40 Ounce
4     Cheez-It Cheddar Cracker


The overall accuracy of the recommendation algorithm on the test data is judged by the Mean Percentile Ranking (MPR) metric. This value is given by,
<center> $\bar{rank} = \frac{\sum_{u,i}r^t_{u,i}rank_{u,i}}{\sum_{u,i}r^t_{u,i}}$ </center>
where $r^t_{u,i}$ represents the number of times a product $i$ is purchased by a user $u$ in the test set, and $rank_{u,i}$ is a percentage denoting how highly a product $i$ is ranked for user $u$ in the recommendation list, where $0%$ denotes a product being recommended at the top of the list. The MPR is bounded by $1$, and the expected MPR of a system that recommends products at random is $0.5$, meaning that a system with a higher value than this is essentially no better than random. This recommendation system received an MPR score of $0.2376$.
<br>
<br>
The implication of MPR is that the recommendation system provide a ranking for every product for each user. However, this process is immensely expensive, so I have modified the method in this notebook to create a list of only the *N* most likely recommendations.

In [9]:
def calculate_mpr(N):
    mpr = []
    test_users = set(df_test.user_id.tolist())
    for user_id in test_users:
        mpr_user = 0

        temp_df = df_test.loc[df_test['user_id'] == user_id]
        products = temp_df.product_id.tolist()
        count = temp_df.count_purchase.tolist()

        recs = recommendations(user_id, N)
        for i in range(len(products)):
            if products[i] in recs:
                rank = recs.index(products[i])
                mpr_user+=(rank/N)*count[i]
        mpr.append(mpr_user/sum(count))
    return mpr

In [16]:
mpr = calculate_mpr(N)
print('Test MPR: {:.4f}'.format(np.mean(mpr)))

Test MPR: 0.2376


## G. Sources
1. [Hu et al., Collaborative Filtering for Implicit Feedback Datasets](http://yifanhu.net/PUB/cf.pdf)
2. [Building a Recommendation System in TensorFlow: Overview](https://cloud.google.com/solutions/machine-learning/recommendation-system-tensorflow-overview)
3. [implicit](https://github.com/benfred/implicit)